In [1]:
import librosa
import numpy as np
from scipy.stats import skew

In [7]:
def compute_audio_freq(y):
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    return [f for f in f0 if str(f) != 'nan']

In [8]:
def compute_audio_skewness(y):
    skewness = skew(y)
    
    return skewness

# Example usage
audio_file = 'output.wav'
y, sr = librosa.load(audio_file)
frequencies = compute_audio_freq(y)
print(frequencies)
skewness = compute_audio_skewness(frequencies)
print(f"Skewness of the audio signal: {skewness}")


[105.64102395561046, 105.03257643014581, 104.42763330455712, 103.82617439498627, 105.03257643014581, 108.1102658399776, 112.57112811964524, 114.5388367788287, 109.99999999999999, 115.86971395503193, 117.89508087899223, 117.89508087899223, 117.89508087899223, 118.57804013471404, 105.64102395561046, 142.65235101161107, 138.59131548843604, 136.9994584594705, 131.5705733911144, 122.05264192746294, 115.86971395503193, 119.26495573315601, 121.34967043801211, 121.34967043801211, 122.05264192746294, 122.75968568932456, 122.75968568932456, 121.34967043801211, 113.87914162255151, 122.05264192746294, 113.22324603078411, 113.22324603078411, 113.22324603078411, 113.87914162255151, 111.9227661312955, 97.99885899543732, 122.75968568932456, 121.34967043801211, 119.26495573315601, 114.5388367788287, 112.57112811964524, 109.99999999999999, 106.86851352689665, 103.82617439498627, 103.22817963382575, 103.22817963382575, 102.63362906904881, 101.45478129445331, 100.87044475251382, 98.56656061031477, 131.570

In [29]:
def spectral_properties(y: np.ndarray, fs: int) -> dict:
    spec = np.abs(np.fft.rfft(y))
    freq = np.fft.rfftfreq(len(y), d=1 / fs)
    spec = np.abs(spec)
    amp = spec / spec.sum()
    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode = freq[amp.argmax()]
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4
    sfm = np.mean(librosa.feature.spectral_flatness(y=y))
    result_d = {
        'mean': mean,
        'sd': sd,
        'median': median,
        'mode': mode,
        'Q25': Q25,
        'Q75': Q75,
        'IQR': IQR,
        'skew': skew,
        'kurt': kurt,
        'sfm': sfm
    }

    return result_d
audio_file = 'output.wav'
y, sr = librosa.load(audio_file)

res = spectral_properties(y, sr)
for key in res.keys():
    print(f"{key:<10} : {res[key]}".format())

frequencies = np.array(compute_audio_freq(y))

sfm = np.mean(librosa.feature.spectral_flatness(y=frequencies))
print(sfm)

mean       : 2324.311783712007
sd         : 2398.3372609301687
median     : 1259.04638671875
mode       : 105.08203125
Q25        : 645.99609375
Q75        : 3437.452880859375
IQR        : 2791.456787109375
skew       : 3.615648780127185
kurt       : 19.223696369191003
sfm        : 0.01844765990972519
0.006725096744753703


c:\Users\Cody\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=255
  warnings.warn(


In [59]:
import numpy as np
import librosa
from scipy.stats import gmean
from scipy.io import wavfile

def spectral_properties(file):
  sample_rate, data = wavfile.read(file)
  y, sr = librosa.load(file)

  #F, f_names = audioFeatureExtraction.stFeatureExtraction(data, sample_rate, 0.050*sample_rate, 0.025*sample_rate);
  print(data.shape)
  print(data[0])
  print(sample_rate)
  print(y.shape)
  print(y[0])
  print(sr)
  spec = np.abs(np.fft.rfft(data))
  magnitude_spectrum = np.abs(np.fft.fft(data))
  max_peak_index = np.argmax(magnitude_spectrum)
  mindom = max_peak_index * sample_rate / len(data)

  freq = np.fft.rfftfreq(len(data), d=1 / sample_rate)
  freq0, = librosa.pyin(y=[float(d) for d in data], fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))

  peakf = np.argmax(freq) # unusued
  amp = spec / spec.sum()
  mean = (freq * amp).sum()
  mean0 = np.nanmean(freq0)
  sd = np.sqrt(np.sum(amp * ((freq0 - mean0) ** 2)))
  amp_cumsum = np.cumsum(amp)
  median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
  mode = freq[amp.argmax()]
  Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
  Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
  IQR = Q75 - Q25
  z = amp - amp.mean()
  w = amp.std()
  skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
  kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4
  spec_flatness = gmean(spec**2)/np.mean(spec**2)

  result_d = {
    'meanfreq': mean/1000,
    'sd': sd/1000,
    'median': median/1000,
    'Q25': Q25/1000,
    'Q75': Q75/1000,
    'IQR': IQR/1000,
    'skew': skew,
    'kurt': kurt,
    # 'sp.ent': F[5].mean(),
    'sfm': spec_flatness,
    'mode': mode/1000,
    # 'meandom': meandom/1000,
    # 'mindom': mindom/1000,
    # 'maxdom': maxdom/1000
    # 'centroid': F[3].mean()/1000,
  }
  return result_d


res = spectral_properties("output.wav")

for key in res.keys():
  print(f"{key:<10} : {res[key]}".format())

(30099,)
0
16000
(41481,)
-3.8365743e-06
22050


ParameterError: Audio data must be floating-point

In [43]:
import numpy as np
from scipy.io import wavfile
from scipy.fft import rfft, rfftfreq

def calculate_frame_dominant_frequency(data, sample_rate):
    # Apply FFT and get frequency bins
    fft_result = rfft(data)
    frequencies = rfftfreq(len(data), 1/sample_rate)
    magnitude = np.abs(fft_result)
    # Find the dominant frequency in the frame
    dominant_frequency = frequencies[np.argmax(magnitude)]
    return dominant_frequency

def calculate_dominant_frequencies_stats(wav_file, frame_size_ms=40, hop_size_ms=20):
    # Read the WAV file
    sample_rate, data = wavfile.read(wav_file)
    
    # If stereo, just use one channel
    if len(data.shape) > 1:
        data = data[:, 0]
    
    # Convert frame size and hop size from milliseconds to samples
    frame_size = int(sample_rate * (frame_size_ms / 1000.0))
    hop_size = int(sample_rate * (hop_size_ms / 1000.0))
    
    # Apply a windowing function to each frame (e.g., Hamming window)
    window = np.hamming(frame_size)
    
    # Initialize an array to hold the dominant frequencies for each frame
    dominant_frequencies = []
    
    # Process each frame of the audio
    for start in range(0, len(data) - frame_size + 1, hop_size):
        frame = data[start:start+frame_size] * window
        dominant_frequency = calculate_frame_dominant_frequency(frame, sample_rate)
        dominant_frequencies.append(dominant_frequency)
    
    # Filter out very low frequencies (e.g., below 20 Hz) to exclude DC component or noise
    dominant_frequencies = [freq for freq in dominant_frequencies if freq >= 20]
    
    # Calculate statistics
    mean_freq = np.mean(dominant_frequencies)/1000
    max_freq = np.max(dominant_frequencies)/1000
    min_freq = np.min(dominant_frequencies)/1000
    
    return mean_freq, max_freq, min_freq

# Example usage:
wav_file_path = 'plaoutput.wav'
mean_freq, max_freq, min_freq = calculate_dominant_frequencies_stats(wav_file_path)
print(f"Mean Dominant Frequency: {mean_freq} kHz")
print(f"Max Dominant Frequency: {max_freq} kHz")
print(f"Min Dominant Frequency: {min_freq} kHz")


Mean Dominant Frequency: 0.7190131578947369 kHz
Max Dominant Frequency: 8.225 kHz
Min Dominant Frequency: 0.025 kHz
